In [22]:
import pandas as pd 
import numpy as np
import string
import json
import pickle
import math

In [26]:
#word_freq_dict will save the frequency of each word in all of the documents
number_of_all_docs = 10
word_freq_dict = {}
#inverted_index_dict will save the inverted index for each word, 
#and also the positions of each word in all the documents
inverted_index_dict = {}
#This dictionary will be like:
"""
inverted_index_dict={
    "word": {"docs": {
                    "doc_id":{
                            "positions": [],
                            "tf": int
                    }
             },
            "df": int
    }
}



"""

stop_words = []

In [4]:
dic1 = {"1":1, "2":22, "3":333}
# # if "4" not in dic1.keys():
# #     dic1["4"] = 4444
# dic1
# stringam = "سلام امیررضا خوبی داداش من خوبم"
f = open("./stop_words.txt", "r")

stop_words_string = f.read()
stop_words_list = stop_words_string.split()
len(stop_words_list)

150

In [5]:
#This method will do some normalization on text files. 
# def normalizer (words_list):
    

In [6]:
#This method will return the positions of a word in a text file (the input is tokens list)
def word_positions (word, tokenized_text):
    positions = [i for i, x in enumerate(tokenized_text) if x == word]
    return positions

#This funcntions will make a list of unique words in a words_list
def different_words (words_list):
    
    different_words_list = []
    for word in words_list:
        if word not in different_words_list:
            different_words_list.append(word)
            
    return different_words_list


In [9]:
#This method will update the inverted index by processing new document
def update_inverted_index (text_file_number, inverted_index_dict, word_freq_dict):
    #Reading the text file 
    
    text_file_name = str(text_file_number)
    file_path = "./sampleDoc/" + text_file_name + ".txt"
    f = open(file_path, "r")

    text = f.read()

    #Remmoving punctuations from the text string
    result = text.translate(str.maketrans('','', string.punctuation))

    #Removin نیم فاصله from the text string
    result = result.replace("\u200c", " ")
#     resutl = result.replace("\u200b", "")
    
    #Splitting the string into seperate words (tokenizing)
    words_list = result.split()
    
    different_words_list = different_words (words_list)
    
    #This loop will update the words frequency dictionary
    for word in words_list:
        if word not in word_freq_dict.keys():
            word_freq_dict[word] = 1
        else:
            word_freq_dict[word] += 1
    
    #This loop will update the inverted index dictionary 
    for word in different_words_list:
        word_positions_list = word_positions (word, words_list) 
        if word not in inverted_index_dict.keys():
            inverted_index_dict[word] = {"docs":{}, "df": 1}
            inverted_index_dict[word]["docs"][text_file_name] = {"positions": word_positions_list, "tf":len(word_positions_list)}
        else:
            inverted_index_dict[word]["docs"][text_file_name] = {"positions": word_positions_list, "tf":len(word_positions_list)}
            inverted_index_dict[word]["df"] += 1
            

In [10]:
# update_inverted_index (1, inverted_index_dict, word_freq_dict)

inverted_index_dict

{}

In [50]:
for i in range (1,11):
    update_inverted_index (i, inverted_index_dict, word_freq_dict)

inverted_index_dict["گزارش"]

{'docs': {'1': {'positions': [8], 'tf': 1},
  '3': {'positions': [16], 'tf': 1},
  '5': {'positions': [13], 'tf': 1},
  '7': {'positions': [23], 'tf': 1},
  '9': {'positions': [0, 40], 'tf': 2},
  '10': {'positions': [55], 'tf': 1}},
 'df': 6}

In [51]:
#We will save the inverted index dictioinary in a pickle file to use it later
inverted_index_dict
a_file = open("data.pkl", "wb")
pickle.dump(inverted_index_dict, a_file)
a_file.close()


In [52]:
#We load the saved pickle file 
a_file = open("data.pkl", "rb")
output = pickle.load(a_file)
print(output)

{'نصوریان': {'docs': {'1': {'positions': [0], 'tf': 1}}, 'df': 1}, 'سرمربی': {'docs': {'1': {'positions': [1, 37], 'tf': 2}}, 'df': 1}, 'تراکتور': {'docs': {'1': {'positions': [2, 14, 46, 80], 'tf': 4}}, 'df': 1}, 'از': {'docs': {'1': {'positions': [3, 24], 'tf': 2}, '3': {'positions': [69, 120, 203], 'tf': 3}, '4': {'positions': [190], 'tf': 1}, '5': {'positions': [59], 'tf': 1}, '6': {'positions': [45, 61, 73, 194], 'tf': 4}, '9': {'positions': [19, 44, 108, 130], 'tf': 4}, '10': {'positions': [3, 60, 88, 167], 'tf': 4}}, 'df': 7}, 'سمتش': {'docs': {'1': {'positions': [4, 25], 'tf': 2}}, 'df': 1}, 'برکنار': {'docs': {'1': {'positions': [5, 26], 'tf': 2}}, 'df': 1}, 'شد': {'docs': {'1': {'positions': [6, 27], 'tf': 2}, '2': {'positions': [4, 170], 'tf': 2}, '3': {'positions': [84], 'tf': 1}, '4': {'positions': [88, 296, 429], 'tf': 3}, '9': {'positions': [102, 134], 'tf': 2}}, 'df': 5}, 'به': {'docs': {'1': {'positions': [7, 58, 64], 'tf': 3}, '2': {'positions': [15, 35, 52, 69, 85, 1

In [53]:


# frequent_words = []

# for word in word_freq_dict:
#     if word_freq_dict[word] > 30:
#         frequent_words.append(word)

# frequent_words

output["تابناک"]["docs"].keys()

dict_keys(['1', '3', '5', '7', '8', '10'])

In [54]:
#This method will sort the documents based on their appearance in results
def sort_docs(query_response): 
    
    query_response_keys = list(query_response.keys())
    query_response_values = list(query_response.values())
    sort_docs_dict = {}
    for doc_list in query_response_values:
        for doc in doc_list:
            if doc not in sort_docs_dict.keys():
                sort_docs_dict[doc] = 1
            else:
                sort_docs_dict[doc] += 1
                
    sort_docs_dict = {k: v for k, v in sorted(sort_docs_dict.items(), key=lambda item: item[1])}
    
    return sort_docs_dict
    
    
#     return list(set(query_response_values[0]) & set(query_response_values[1])) 
    print (token_combinations_list[0][0][1])

#This method will process the user's query and prints response for him
def query ():
    
    query_response = {}
    query = input()
    query_tokens = query.split()
    
    #This counter is to check whether there is any match with this query or not
    counter = 0
    
    for token in query_tokens:
        if token in output.keys():
            counter += 1
            query_response[token] = list(output[token]["docs"].keys())
            
    if counter == 0:
        print("No match found!")
    else:
#         print("salam")
#         print(query_response)
        ans = sort_docs(query_response) 
        print(ans)
    


In [55]:
# query()
output['گزارش']


{'docs': {'1': {'positions': [8], 'tf': 1},
  '3': {'positions': [16], 'tf': 1},
  '5': {'positions': [13], 'tf': 1},
  '7': {'positions': [23], 'tf': 1},
  '9': {'positions': [0, 40], 'tf': 2},
  '10': {'positions': [55], 'tf': 1}},
 'df': 6}

In [56]:
query()

گزارش
{'1': 1, '3': 1, '5': 1, '7': 1, '9': 1, '10': 1}


In [59]:
#This method will calculate the tf-idf weight for given term, and document
#Instead of storing the weights we will calculate them based on tf and idf, because it will occupy lower space 
def calculate_weight (term, document_number, inverted_index_dict):
    
    print(term)
    print(document_number)
    print(inverted_index_dict[term])
    # !!! document number is a string e.g. "x"
    
    tf_td = inverted_index_dict[term]["docs"][document_number]['tf']
    idf_t = math.log10(number_of_all_docs/inverted_index_dict[term]['df'])
    
    if tf_td > 0:
        w_td = (1 + math.log10(tf_td)) * idf_t
    else:
        w_td = 0
    
    return w_td

In [62]:
score = calculate_weight("گزارش", "2", inverted_index_dict)
score

گزارش
2
{'docs': {'1': {'positions': [8], 'tf': 1}, '3': {'positions': [16], 'tf': 1}, '5': {'positions': [13], 'tf': 1}, '7': {'positions': [23], 'tf': 1}, '9': {'positions': [0, 40], 'tf': 2}, '10': {'positions': [55], 'tf': 1}}, 'df': 6}


KeyError: '2'

In [ ]:
def calculate_cosine_score (query):
    